In [ ]:
import sys
import mysql.connector
import numpy as np
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id = 1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)
#TODO: inciroirar el año desde el server al variable year
clientes=pd.read_sql("""select  SUM(internal_orders.total) as total,COUNT(internal_orders.id) as pi,customers.customer,customers.clave from customers inner join internal_orders
                     on internal_orders.customer_id = customers.id
                     where internal_orders.date > '2024-01-01' 
                     group by customers.customer,customers.clave order by total """,cnx)

#traer datos de los pedidos
pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.customer
from (
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
                    order by internal_orders.total
     
     """,cnx)

objetivo=pd.read_sql("""select * 
                     from settings""",cnx)['objetivo_anual'].values[0]


nordenes=len(pedidos)
df=pedidos[['date']]

tc=pd.read_sql('select * from coins where id=13 ',cnx)['exchange_sell'].values[0]
writer = pd.ExcelWriter('storage/report/rango_ventas_pi1.xlsx', engine='xlsxwriter')
workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
     'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00'})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':13})

blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    })


blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'
    })

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#356e31',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#3e5585',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})


import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
rangos=[
        [0,50],
        [51,100],
        [101,200],
        [301,400],
        [401,500],
        [500,600],
        [601,700],
        [701,800],
        [801,900],
        [901,1000],
        [1000,2000],
        [2001,3000],
        [3001,99000]
        ]
dias_transcurridos=date-datetime.date(int(year), 1, 1)
#Columna para filtrar por fechas
pedidos['date']=pd.to_datetime(pedidos['date'])
pedidos=pedidos.loc[pedidos['date']>year+'-01-01']
df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------
worksheet.merge_range('B2:F2', 'CUENTAS POR COBRAR REPORTE 3/8', negro_b)
worksheet.merge_range('B3:F3', 'RANGO DE VENTAS POR CLIENTE', negro_s)

worksheet.write('H2', 'AÑO', negro_b)
worksheet.write('I2', year, negro_b)
worksheet.merge_range('J2:K3', """FECHA DEL REPORTE
DD/MM/AAAA""", negro_b)
worksheet.merge_range('J4:K4', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})
#Cual es el rango con mas filas?
max_rows=0
n=0
for i in rangos:
    li=i[0]*1000
    ls=i[1]*1000
    target_pedidos=pedidos.loc[(pedidos['total']>=li)&(pedidos['total']<=ls)]
    if(len(target_pedidos)>max_rows):
         max_rows=len(target_pedidos)
#Tablas de cada rango en fila
for i in rangos:
    
    print('de',i[0],'a',i[1])
    li=i[0]*1000
    ls=i[1]*1000
    target_pedidos=pedidos.loc[(pedidos['total']>=li)&(pedidos['total']<=ls)]
    if(len(target_pedidos)>0):
        worksheet.merge_range(6,n*4+1,6,n*4+4,'DE '+str(i[0])+' A '+str(i[1]), blue_header_format)
        worksheet.write(7,n*4+1,'NO.', blue_header_format)
        worksheet.write(7,n*4+2,'CLIENTE', blue_header_format)
        worksheet.write(7,n*4+3,'PI', blue_header_format)
        worksheet.write(7,n*4+4,'TOTAL', blue_header_format)
        for j in range(len(target_pedidos)):
            print(target_pedidos['total'].values[j],target_pedidos['customer'].values[j])
            worksheet.write(8+j,n*4+1,target_pedidos['clave'].values[j].replace(' ',''), blue_content)
            worksheet.write(8+j,n*4+2,target_pedidos['customer'].values[j], blue_content)
            worksheet.write(8+j,n*4+3,target_pedidos['invoice'].values[j], blue_content_unit)
            worksheet.write(8+j,n*4+4,target_pedidos['total'].values[j], blue_content)
        for j in np.arange(len(target_pedidos),max_rows,1):
            worksheet.write(8+j,n*4+1,' ', blue_content)
            worksheet.write(8+j,n*4+2,' ', blue_content)
            worksheet.write(8+j,n*4+3,' ', blue_content)
            worksheet.write(8+j,n*4+4,' ', blue_content)

        worksheet.merge_range(8+max_rows,n*4+1,8+max_rows,n*4+3,'TOTAL', blue_header_format)
        worksheet.write(8+max_rows,n*4+4,target_pedidos['total'].sum(), blue_header_format)

        n=n+1

 
# Tablas inferiores
worksheet.write(11+max_rows,1,'CANATIDAD', blue_header_format)
worksheet.write(11+max_rows,2,'RANGO', blue_header_format)
worksheet.write(11+max_rows,3,'$', blue_header_format)
worksheet.write(11+max_rows,4,'% EN PI', blue_header_format)
worksheet.write(11+max_rows,5,'% EN $', blue_header_format)

#Tabla de resumen de rangos
n=0
for i in rangos:
    li=i[0]*1000
    ls=i[1]*1000
    target_pedidos=pedidos.loc[(pedidos['total']>=li)&(pedidos['total']<=ls)]
    
    if(len(target_pedidos)>0):
        worksheet.write(12+max_rows+n,1,len(target_pedidos),blue_content_unit)
        worksheet.write(12+max_rows+n,2,'DE '+str(i[0])+' A '+str(i[1]),blue_content)
        worksheet.write(12+max_rows+n,3,target_pedidos['total'].sum(),blue_content)
        worksheet.write(12+max_rows+n,4,str(round(len(target_pedidos)*100/len(pedidos),2))+'%',blue_content_unit)
        worksheet.write(12+max_rows+n,5,str(round(target_pedidos['total'].sum()*100/pedidos['total'].sum(),2))+'%',blue_content_unit)
        n=n+1



worksheet.merge_range(11+max_rows,7,11+max_rows,8,'SUMA DE PEDIDOS AL AÑO', blue_header_format)
worksheet.write(11+max_rows,9,str(len(pedidos)),blue_content_bold)
worksheet.merge_range(12+max_rows,7,12+max_rows,8,'VENTAS EN MONEDA NACIONAL', blue_header_format)
worksheet.write(12+max_rows,9,pedidos['total'].sum(),blue_content_bold)
#AGRANDAR CPLUMNAS
worksheet.set_column('A:A',15)
worksheet.set_column('C:C',33)
worksheet.set_column('E:E',23)
worksheet.set_column('F:F',20)

worksheet.set_column('G:G',33)

worksheet.set_column('I:I',23)
worksheet.set_column('J:J',20)
worksheet.set_column('K:K',33)

worksheet.set_column('N:N',20)
worksheet.set_column('O:O',33)

worksheet.set_column('R:R',20)
worksheet.set_column('S:S',33)

#worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
worksheet.set_landscape() 
#Hoja de graficas

# worksheet_charts = workbook.add_worksheet("Gráficas")
# worksheet_charts.merge_range('B2:F2', 'CUENTAS POR COBRAR REPORTE 3/8', negro_b)
# worksheet_charts.merge_range('B3:F3', 'RANGO DE VENTAS POR CLIENTE', negro_s)

# worksheet_charts.write('H2', 'AÑO', negro_b)
# worksheet_charts.write('I2', year, negro_b)
# worksheet_charts.merge_range('J2:K3', """FECHA DEL REPORTE
# DD/MM/AAAA""", negro_b)
# worksheet_charts.merge_range('L2:L3', date, negro_b)
# worksheet_charts.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})

# # Create a new chart object.
# chart = workbook.add_chart({'type': 'pie','subtype': 'Monto por rango'})

# # Add a series to the chart.
# chart.add_series({'values': '=Sheet1!$D$'+str(12+max_rows)+':$D$'+str(12+max_rows+ len(rangos)),
#                   'categories': '=Sheet1!$C$'+str(12+max_rows)+':$C$'+str(12+max_rows+ len(rangos)),
#                   'percentage': True,
#                     'leader_lines': True,
#                     'position': 'best_fit',
#                     'data_labels': {
#                     'value': True,
#                     'font': {'color': 'gray','size': 10}
#                 }})


# # Insert the chart into the worksheet.
# worksheet_charts.insert_chart('B5', chart,{'x_scale': 2.15, 'y_scale': 1.35})


# # Create a new chart object.
# chart = workbook.add_chart({'type': 'pie','subtype': 'No. PI por rango'})
# chart.set_title({
#     'name': 'Np. PI por rango'})
# # Add a series to the chart.
# chart.add_series({'values': '=Sheet1!$B$'+str(12+max_rows)+':$B$'+str(12+max_rows+ len(rangos)),
#                   'categories': '=Sheet1!$C$'+str(12+max_rows)+':$C$'+str(12+max_rows+ len(rangos)),
#                    'categories': '=Sheet1!$C$'+str(12+max_rows)+':$C$'+str(12+max_rows+ len(rangos)),
#                   'percentage': True,
#                     'leader_lines': True,
#                     'position': 'best_fit',
#                     'data_labels': {
#                     'value': True,
#                     'font': {'color': 'gray','size': 10}}
#                   })


# # Insert the chart into the worksheet.
# worksheet_charts.insert_chart('B25', chart,{'x_scale': 2.15, 'y_scale': 1.35})




workbook.close()
                

de 0 a 50
49221.12 SKYTEX MEXICO
37416.96 COMERCIALIZADORA NARECI S.A. DE C.V.
0.0 AIR ARQUITECTURA INTEGRAL RAMONETTISA SA DE CV
43848.0 AUTOPARTES IMPORTADAS DE OCCIDENTE SA DE CV
38976.0 CARNES EL CORRALITO, S. DE R.L. DE C.V.
5336.0 TIENDAS TRES B S.A. DE C.V.
7440.24 MADERERIA LA ARBOLEDA
28565.0 BLUE MARBLE TECHNOLOGIES
32596.0 SUSPENSION Y DIRECCION
34684.0 THE FLAVOR TASTE SA DE CV
17915.04 EDELVIVES MEXICO
16588.0 MANUFACTURA DE PRODUCTOS METALICOS Y ACABADOS FCA
893.2 CINDY MEZA GUERRERO
8514.4 CINDY MEZA GUERRERO
24766.0 COMERCIALIZADORA PANJUM
0.0 JOHANN ANTONIO HERREÑO ARIAS
30044.0 R&T INNOVACION INDUSTRIAL SA DE CV
31120.48 BLUE MARBLE TECHNOLOGIES
7540.0 WURTH MEXICO
de 51 a 100
80997.0 CROWN GAMING MEXICO
51620.0 EDELVIVES MEXICO
64913.6 MORTERO STUDIO
79924.0 TIENDAS TRES B S.A. DE C.V.
64728.0 TIENDAS TRES B S.A. DE C.V.
85677.6 MORTERO STUDIO
55557.04 CROWN GAMING MEXICO
95304.44 AUTOMOTRIZ SERVA, S.A. DE C.V
87464.0 EQUIPOS ESPECIALIZADOS EN RENTA NG
68574.56 ELAST

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

In [20]:
clientes=pd.read_sql("""select  SUM(internal_orders.total) as total,customers.customer from customers inner join internal_orders
                     on internal_orders.customer_id = customers.id
                     where internal_orders.date > '01-01-2024' 
                     group by customers.customer order by total """,cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_10852\497557552.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clientes=pd.read_sql("""select  SUM(internal_orders.total) as total,customers.customer from customers inner join internal_orders


In [8]:
data

{'Enero': {'GUTIERREZ\tABADIA\tALEJANDRO': 0,
  'HERREÑO\tCORRALES\tNELSON ANDRES': 0,
  'HERREÑO\tCORRALES\tCHRISTIAN LEONARDO': 0,
  'HERREÑO\tCORRALES\tFREDY ALEXANDER': 0,
  'WUNSCH\tCERON\tJAIR': 0,
  'GUERRERO\tSERRANO\tEDUARDO': 0,
  'SOLIS\tDE LA TORRE\tANTONIO': 0,
  'ROMO\tRAMIREZ\tHADRIEL': 0,
  'RAMIREZ\tRODRIGUEZ \tALBERTO': 0,
  'ROMERO\tZAVALA\tMIGUEL ERNESTO': 0,
  'PEÑA\tBARRIOS\tJOSE LUIS': 0,
  'TREJO\tJUAREZ\tALEJANDRO': 0,
  'BUSTAMANTE\tFERNANDEZ\tFERNANDO': 0,
  'PAREDES\tTORRES\tGUILLERMO': 0,
  'AVILA JIMENEZ OSCAR ULISES': 0,
  'GARCIA\tCASTRO\tDORIAN': 0,
  'MUÑOZ\tFLORES\tARTURO': 0,
  'OLGUIN\tMARISCAL\tALEJANDRO': 0,
  'MORALES\tGALLEGOS\tIVAN': 0,
  'ORDOÑEZ\tJUAREZ\tALEJANDRA': 0,
  'CASA\tCASA\tCASA': 0,
  'HERREÑO\tMORENO\tNELSON ERNESTO': 0,
  'GASPAR ISLAS SANTOYO': 0},
 'Febrero': {'GUTIERREZ\tABADIA\tALEJANDRO': 0,
  'HERREÑO\tCORRALES\tNELSON ANDRES': 0,
  'HERREÑO\tCORRALES\tCHRISTIAN LEONARDO': 0,
  'HERREÑO\tCORRALES\tFREDY ALEXANDER': 0,
  'WU

In [7]:
pedidos_interes=pedidos_interes.assign(exceso=lambda x: x.saldo -x.total )

In [23]:
pxv['date']=pd.to_datetime(pxv['date'])
worksheet.write(7+i,mes+1, len(pxv.loc[(pxv['date']<str(year)+'-'+str(mes+2)+'-01')&(pxv['date']>=str(year)+'-'+str(mes+2)+'-01')]), blue_content)

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_24124\2599723469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pxv['date']=pd.to_datetime(pxv['date'])


0

In [22]:
len(pxv.loc[(pxv['date']<str(year)+'-'+str(mes+2)+'-01')&(pxv['date']>=str(year)+'-'+str(mes+2)+'-01')])

0

In [ ]:
for i in range(len(vendedores)):
    seller_id=vendedores['id'].values[i]
    pxv=pedidos.loc[pedidos['seller_id']==seller_id]
    pedidos.loc[(pedidos['date']<year+'-02-01')&(pedidos['date']>=year+'-01-01')]
    print(len(pxv))

3
64
0
30
0
0
9
0
1
0
5
2
64
20
8
9
2
17
30
27
1
0
1


In [5]:
len(pedidos)

298

In [14]:
year

'2024'

In [12]:
pedidos['date']=pd.to_datetime(pedidos['date'])

In [16]:
pedidos.loc[(pedidos['date']<year+'-02-01')&(pedidos['date']>=year+'-01-01')]

,id,invoice,date,customer_id,seller_id,date_delivery,instalation_date,shipment,customer_shipping_address_id,coin_id,...,ret,tasa,description,category,clave,alias,exchange_sell,coin,symbol,code
235,383,3150,2024-01-05,670,15,2024-02-09,2024-02-09,Sí,271,1,...,NaN,0.0,PF+SI,Integracion,375,COMERCIALIZADORA PANJUM,1.0,NACIONAL,$,MN
236,385,3151,2024-01-10,424,2,2024-01-17,2024-01-17,No,29,1,...,NaN,0.0,Servicio indirecto SI,Servicios,18,SKYTEX,1.0,NACIONAL,$,MN
237,390,3152,2024-01-19,317,14,2024-02-28,2024-03-06,Sí,273,1,...,NaN,0.0,PF+SI,Integracion,219,NARECI,1.0,NACIONAL,$,MN
238,391,3153,2024-01-19,671,2,2024-03-01,2024-03-08,Sí,274,1,...,NaN,0.0,PF+SI,Integracion,376,RAMONETTISA,1.0,NACIONAL,$,MN
239,392,3154,2024-01-25,672,14,2024-03-19,2024-03-26,Sí,276,1,...,NaN,0.0,PF+SI,Integracion,362,INDUSTRIAS UNIDAS,1.0,NACIONAL,$,MN
240,393,3155,2024-01-25,672,14,2024-03-26,2024-04-02,Sí,277,1,...,NaN,0.0,PF+SI,Integracion,362,INDUSTRIAS UNIDAS,1.0,NACIONAL,$,MN
241,394,3156,2024-01-25,672,14,2024-04-02,2024-04-02,Sí,278,1,...,NaN,0.0,PF+SI,Integracion,362,INDUSTRIAS UNIDAS,1.0,NACIONAL,$,MN
242,395,3157,2024-01-25,673,20,2024-03-11,2024-03-18,Sí,279,1,...,NaN,0.0,PF+SI,Integracion,377,SOLUCIONES EN ARRENDAMIENTO,1.0,NACIONAL,$,MN
243,396,3158,2024-01-26,621,5,2024-03-21,2024-03-11,Sí,280,1,...,NaN,0.0,PF+SI,Integracion,329,LIDERSOL,1.0,NACIONAL,$,MN
244,397,3159,2024-01-26,659,5,2024-03-21,2024-03-10,Sí,248,1,...,NaN,0.0,PF+SI,Integracion,363,BEE BAGS,1.0,NACIONAL,$,MN


In [8]:
pedidos['date']

0      2023-04-04
1      2023-04-04
2      2023-04-09
3      2023-04-09
4      2023-04-10
          ...    
293    2024-04-19
294    2024-04-26
295    2024-04-29
296    2024-04-30
297    2024-05-06
Name: date, Length: 298, dtype: object

In [1]:
4*1.6

6.4

In [1]:
8/19

0.42105263157894735

In [2]:
for i in range(9):
    print(i/9)

0.0
0.1111111111111111
0.2222222222222222
0.3333333333333333
0.4444444444444444
0.5555555555555556
0.6666666666666666
0.7777777777777778
0.8888888888888888
